In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn import metrics, tree, ensemble
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold

In [ ]:
wines=pd.read_csv('data/wine.data', names=["Clazz", "Alcohol", "Malic acid", "Ash","Alcalinity of ash", "Magnesium",
 "Total phenols",
"Flavanoids",
 "Nonflavanoid phenols",
 "Proanthocyanins",
"Color intensity",
"Hue",
"OD280/OD315 of diluted wines",
"Proline"])
wines.head()

In [ ]:
y=wines.Clazz
X=wines.drop('Clazz', axis=1)

## Run Nested CV to Compare Base, Bagging, and Boosting Classifiers

In [ ]:
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)


def evaluate_classifier(model, features, target):
    tic=time.time()
    res = cross_val_score(model, features, target, cv=outer_cv, n_jobs=5, scoring='accuracy')
    print(f"The mean accuracy of the outer cross validation: "
      f"{res.mean():.3f} +/- {res.std():.3f}") 
    print(f"Time for Evaluation: {(time.time()-tic):.3f} seconds.")

### Bagging
For this experiment we do not optimize hyperparameters of the base algorithm 
(usually these hyperparameters have to be optimized as well).

In [ ]:
dtc=tree.DecisionTreeClassifier()
evaluate_classifier(dtc, X, y)

In [ ]:
bagging_clf = ensemble.BaggingClassifier(base_estimator=dtc,
                            #n_estimators=200,
                            #max_samples=.5,
                            max_features=1.0,
                            n_jobs=5)

bagging_grid = {
    "n_estimators": [10, 20, 50, 100, 200, 500], 
    "max_samples": np.array(range(1,11))/10.0
}

# hyperparameter selection
bagging_grid_cv = GridSearchCV(
    estimator=bagging_clf, param_grid=bagging_grid, cv=inner_cv, n_jobs=5, scoring='accuracy')

evaluate_classifier(bagging_grid_cv, X, y)

In [ ]:
bagging_grid_cv.fit(X,y)
bagging_grid_cv.best_params_

### Boosting

In [ ]:
dtc1=tree.DecisionTreeClassifier(max_depth=1) # for this experiment we explicitly impede the trees performance by allowing only small depth
evaluate_classifier(dtc1, X, y)

In [ ]:
ada_clf = ensemble.AdaBoostClassifier(base_estimator=dtc1)

ada_grid={
    'n_estimators':[10, 20, 50, 100, 200, 500],
    'learning_rate':[0.1, 0.2, 0.5, 1.0, 2.0]
}
ada_grid_cv = GridSearchCV(
    estimator=ada_clf, param_grid=ada_grid, cv=inner_cv, n_jobs=5, scoring='accuracy')

evaluate_classifier(ada_grid_cv, X, y)

In [ ]:
ada_grid_cv.fit(X,y)
ada_grid_cv.best_params_

Note that we used a non-binary classification problem in this demo. The steps of AdaBoost can easily be adapted to non-binary problems.